In [1]:
from azure.identity import AzureCliCredential

credential = AzureCliCredential()
from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential



credential = AzureCliCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="6fab868f-e7ff-4000-b366-b5d0a27cc581",
    resource_group_name="shafi-iat",
    workspace_name="shafi-iat"
)


In [138]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
model = Model(
    path="./GBM_model_python_1749296476765_1.zip",  # Local path to your model
    type=AssetTypes.CUSTOM_MODEL,
    name="h2o-model",  # Desired model name in Azure ML
    description="My custom model registered from local file"
)
registered_model = ml_client.models.create_or_update(model)
print(f"Model registered: {registered_model.name}, Version: {registered_model.version}")

Model registered: h2o-model, Version: 4


In [ ]:
# model_name = "my_model"

# models = list(ml_client.models.list(name=model_name))
# print(f"\nAll registered versions of '{model_name}':")
# for m in models:
#     print(f"  - Version {m.version}: asset ID {m.id}")
    
# latest = max(models, key=lambda x: int(x.version))
# print(f"\n✅ Latest version is {latest.version}")


All registered versions of 'my_model':
  - Version 2: asset ID /subscriptions/6fab868f-e7ff-4000-b366-b5d0a27cc581/resourceGroups/iat/providers/Microsoft.MachineLearningServices/workspaces/shafi4/models/my_model/versions/2
  - Version 1: asset ID /subscriptions/6fab868f-e7ff-4000-b366-b5d0a27cc581/resourceGroups/iat/providers/Microsoft.MachineLearningServices/workspaces/shafi4/models/my_model/versions/1

✅ Latest version is 2


In [2]:
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [139]:
from azure.ai.ml.entities import Environment
name="h2o-env"


env = Environment(
    name=name,
    version="2",  # bump version manually
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04"
)
ml_client.environments.create_or_update(env)


ml_client.environments.create_or_update(env)
# 1️⃣ Fetch all versions of the environment

env_versions = list(ml_client.environments.list(name=name))

print(f"Registered versions for environment '{name}':")
for env in env_versions:
    print(f" • Version {env.version}")

# 2️⃣ Find the latest version by numeric comparison
latest_env = max(env_versions, key=lambda e: int(e.version))
print(f"\n✅ Latest version is {latest_env.version}")


Registered versions for environment 'h2o-env':
 • Version 2
 • Version 1

✅ Latest version is 2


In [140]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "iat-endpiont-v2"

In [141]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint_result = ml_client.begin_create_or_update(endpoint).result()

print(
    f"Endpint {endpoint_result.name} provisioning state: {endpoint_result.provisioning_state}"
)

Endpint iat-endpiont-v2 provisioning state: Succeeded


In [142]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "iat-endpiont-v2" with provisioning state "Succeeded" is retrieved


In [143]:
from azure.ai.ml.entities import Model


model = ml_client.models.get(name="h2o-model", version="4")

In [144]:

from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    Environment,
    CodeConfiguration
)
code_config = CodeConfiguration(
    code=".",              # Folder containing score.py
    scoring_script="score.py"  # Your scoring script
)

# Step 3: Attach environment and code config to deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment="h2o-env:2",
    code_configuration=code_config,
    instance_type="Standard_E2s_v3",
    instance_count=1,
)

In [145]:
# Create the deployment
blue_deployment_results = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

print(f"Deployment state: {blue_deployment_results.provisioning_state}")

Check: endpoint iat-endpiont-v2 exists
Uploading testiat (5.18 MBs): 100%|██████████| 5184193/5184193 [00:03<00:00, 1609683.40it/s]




......................................................................Deployment state: Succeeded


In [67]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [68]:
%%writefile {deploy_dir}/sample-request.json
{
  "data": [
    {
      "sepal_length": 6.7,
      "sepal_width": 3.1,
      "petal_length": 4.7,
      "petal_width": 1.5
    }
  ]
}

Writing ./deploy/sample-request.json


In [150]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name="iat-endpiont",
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

'"[{\\"predict\\":\\"versicolor\\",\\"setosa\\":0.001944083,\\"versicolor\\":0.9898267126,\\"virginica\\":0.0082292044}]"'

In [149]:
import urllib.request
import json

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "data": [
    {
      "sepal_length": 6.7,
      "sepal_width": 3.1,
      "petal_length": 4.7,
      "petal_width": 1.5
    }
  ]
}

body = str.encode(json.dumps(data))

url = 'https://iat-endpiont-v2.southcentralus.inference.ml.azure.com/score'
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = '7f8fl8kAMg4E0TOnptvt6wVWLs5JXZtqYL2bubFIR1Yekj9bDCC8JQQJ99BFAAAAAAAAAAAAINFRAZML33hg'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

The request failed with status code: 404
content-type: text/plain
content-length: 506
date: Mon, 09 Jun 2025 19:06:46 GMT
server: azureml-frontdoor
x-request-id: 11515028-364d-44c2-8e5a-3dd1d5eae79d
connection: close


No valid deployments to route to. Please check that the endpoint has at least one deployment with positive weight values or use a deployment specific header to route. [https://docs.microsoft.com/en-us/azure/machine-learning/how-to-safely-rollout-managed-endpoints#deploy-a-new-model-but-send-it-no-traffic-yet]
Please check this guide to understand why this error code might have been returned 
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints#http-status-codes

